Refs: 
    https://github.com/bikz05/ipython-notebooks/blob/master/computer-vision/displaying-video-in-ipython-notebook.ipynb

In [1]:
%qtconsole

In [21]:
# one-time prep work
import numpy as np
import matplotlib.pyplot as plt
# set matplotlib to render in this window, with the specified figure size
%matplotlib inline
import pylab
#pylab.rcParams['figure.figsize'] = (12.0, 8.0)
import cv2


Populating the interactive namespace from numpy and matplotlib


In [12]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

################################TODO##################
from imutils import paths


from IPython.display import clear_output
%pylab inline 
pylab.rcParams['figure.figsize'] = (12.0, 8.0)

import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import scipy.ndimage as sciimg
import os

Populating the interactive namespace from numpy and matplotlib


In [17]:
#Video/Data Paths

nonePath = 'Videos/Data/None/None_true.mp4'
itemPath = 'Videos/Data/Tear/Tear_Ww.mp4'


In [4]:
def drawGridLines(frame, line_color=(0, 255, 0), thickness=1, type_=cv2.LINE_AA, pxstep=100):
    x = pxstep
    y = pxstep
    while x < frame.shape[1]:
        cv2.line(frame, (x, 0), (x, frame.shape[0]), color=line_color, lineType=type_, thickness=thickness)
        x += pxstep
    while y < frame.shape[0]:
        cv2.line(frame, (0, y), (frame.shape[1], y), color=line_color, lineType=type_, thickness=thickness)
        y += pxstep
        

In [24]:
def videoToFrameLabel(vidpath, outDir='train_images/'):
    #Read Video Frame By Frame
    vid = cv2.VideoCapture(vidpath)

    # Put the code in try-except statements
    # Catch the keyboard exception and 
    # release the camera device and 
    # continue with the rest of code.
    try:
        frame_no = 0
        prefix = outDir + vidpath.split('/')[-1].split('.')[0]
        while(True):
            frame_no+=1 
            # Capture frame-by-frame
            ret, frame = vid.read()
            if not ret:
                # Release the Video Device if ret is false
                vid.release()
                # Message to be displayed after releasing the device
                print("Released Video Resource")
                break
            # Convert the image from OpenCV BGR format to matplotlib RGB format
            # to display the image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            outPath = prefix + '-' + str(frame_no) +'.png'
            print("[INFO] Writing "+outPath)
            cv2.imwrite(outPath,frame)
    except KeyboardInterrupt:
        # Release the Video Device
        vid.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource")


In [26]:
itemPath = 'Videos/Data/Tear/Tear_Kha.mp4'
videoToFrameLabel(itemPath, outDir='train_images/tear/')

[INFO] Writing train_images/tear/Tear_Kha-1.png
[INFO] Writing train_images/tear/Tear_Kha-2.png
[INFO] Writing train_images/tear/Tear_Kha-3.png
[INFO] Writing train_images/tear/Tear_Kha-4.png
[INFO] Writing train_images/tear/Tear_Kha-5.png
[INFO] Writing train_images/tear/Tear_Kha-6.png
[INFO] Writing train_images/tear/Tear_Kha-7.png
[INFO] Writing train_images/tear/Tear_Kha-8.png
[INFO] Writing train_images/tear/Tear_Kha-9.png
[INFO] Writing train_images/tear/Tear_Kha-10.png
[INFO] Writing train_images/tear/Tear_Kha-11.png
[INFO] Writing train_images/tear/Tear_Kha-12.png
[INFO] Writing train_images/tear/Tear_Kha-13.png
[INFO] Writing train_images/tear/Tear_Kha-14.png
[INFO] Writing train_images/tear/Tear_Kha-15.png
[INFO] Writing train_images/tear/Tear_Kha-16.png
[INFO] Writing train_images/tear/Tear_Kha-17.png
[INFO] Writing train_images/tear/Tear_Kha-18.png
[INFO] Writing train_images/tear/Tear_Kha-19.png
[INFO] Writing train_images/tear/Tear_Kha-20.png
[INFO] Writing train_images/t

[INFO] Writing train_images/tear/Tear_Kha-169.png
[INFO] Writing train_images/tear/Tear_Kha-170.png
[INFO] Writing train_images/tear/Tear_Kha-171.png
[INFO] Writing train_images/tear/Tear_Kha-172.png
[INFO] Writing train_images/tear/Tear_Kha-173.png
[INFO] Writing train_images/tear/Tear_Kha-174.png
[INFO] Writing train_images/tear/Tear_Kha-175.png
[INFO] Writing train_images/tear/Tear_Kha-176.png
[INFO] Writing train_images/tear/Tear_Kha-177.png
[INFO] Writing train_images/tear/Tear_Kha-178.png
[INFO] Writing train_images/tear/Tear_Kha-179.png
[INFO] Writing train_images/tear/Tear_Kha-180.png
[INFO] Writing train_images/tear/Tear_Kha-181.png
[INFO] Writing train_images/tear/Tear_Kha-182.png
[INFO] Writing train_images/tear/Tear_Kha-183.png
[INFO] Writing train_images/tear/Tear_Kha-184.png
[INFO] Writing train_images/tear/Tear_Kha-185.png
[INFO] Writing train_images/tear/Tear_Kha-186.png
[INFO] Writing train_images/tear/Tear_Kha-187.png
[INFO] Writing train_images/tear/Tear_Kha-188.png


In [14]:
#Read Video Frame By Frame
vidpath = itemPath
# You can also give path to the video file
vid = cv2.VideoCapture(vidpath)

# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
try:
    frame_no = 0
    prefix = vidpath.split('/')[-1].split('.')[0]
    while(True):
        frame_no+=1 
        # Capture frame-by-frame
        ret, frame = vid.read()
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        drawGridLines(frame, pxstep=200)
        # Turn off the axis
        # Title of the window
        title("Input Stream Frame_No: "+str(frame_no))
        # Display the frame
        imshow(frame)
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")

Released Video Resource
